In [2]:
import json
import requests

import pandas as pd
import geopandas as gpd

from datetime import datetime

In [ ]:
def extract_forces_api(type: str = None):

    base_url = 'https://data.police.uk/api/forces'
    if type == None:
        url = base_url
    elif 'leicestershire' == type.lower():
        url = base_url + '/' + type.lower()
    elif 'leicestershire/people' == type.lower():
        url = base_url + '/' + type.lower()

    r = requests.get(url)
    list_of_dict = json.loads(r.content)
    return list_of_dict


In [ ]:
extract_forces_api()

[{'id': 'avon-and-somerset', 'name': 'Avon and Somerset Constabulary'},
 {'id': 'bedfordshire', 'name': 'Bedfordshire Police'},
 {'id': 'cambridgeshire', 'name': 'Cambridgeshire Constabulary'},
 {'id': 'cheshire', 'name': 'Cheshire Constabulary'},
 {'id': 'city-of-london', 'name': 'City of London Police'},
 {'id': 'cleveland', 'name': 'Cleveland Police'},
 {'id': 'cumbria', 'name': 'Cumbria Constabulary'},
 {'id': 'derbyshire', 'name': 'Derbyshire Constabulary'},
 {'id': 'devon-and-cornwall', 'name': 'Devon & Cornwall Police'},
 {'id': 'dorset', 'name': 'Dorset Police'},
 {'id': 'durham', 'name': 'Durham Constabulary'},
 {'id': 'dyfed-powys', 'name': 'Dyfed-Powys Police'},
 {'id': 'essex', 'name': 'Essex Police'},
 {'id': 'gloucestershire', 'name': 'Gloucestershire Constabulary'},
 {'id': 'greater-manchester', 'name': 'Greater Manchester Police'},
 {'id': 'gwent', 'name': 'Gwent Police'},
 {'id': 'hampshire', 'name': 'Hampshire Constabulary'},
 {'id': 'hertfordshire', 'name': 'Hertford

# Crime related

In [ ]:
def extract_street_level_crimes(lat: float = None, lng: float = None, date: str = None, poly: tuple = None, crime_type: str = None):
    """
    Params:
    lat - Latitude of the requested crime area
    lng - Longitude of the requested crime area
    date - Optional. "YYYY-MM" Limit results to a specific month. The latest month will be shown by default
    OR: 
    poly - Limit results to a specific area. The lat/lng pairs which define the boundary of the custom area

    You can also specify the type of crime:
    crime_type - The type of crime. Options are: None, anti-social-behavior, burglary
    """
    # Get current year
    current_year = datetime.now().year

    # Get current month
    current_month = datetime.now().month

    if crime_type == None:
        base_url = "https://data.police.uk/api/crimes-street/all-crime?"
    elif crime_type != None:
        base_url = f"https://data.police.uk/api/crimes-street/{crime_type}?"

    if date == None:
        if lat and lng != None:
            url = base_url + f"lat={lat}&lng={lng}&date={current_year}-{current_month}"
        elif poly != None:
            url = base_url + f"poly={poly[0][0]}:{poly[0][1]},{poly[1][0]}:{poly[1][1]},{poly[2][0]},{poly[2][1]}&date={current_year}-{current_month}"
        else:
            raise ValueError(f'Specify the lat/lng, you passed {lat, lng} or poly {poly}')
    
    elif date != None:
        if lat and lng != None:
            url = base_url + f"lat={lat}&lng={lng}&date={date}"
        elif poly != None:
            url = base_url + f"poly={poly[0][0]}:{poly[0][1]},{poly[1][0]}:{poly[1][1]},{poly[2][0]}:{poly[2][1]}&date={date}"
        else:
            raise ValueError(f'Specify the lat/lang, you passed {lat, lng} or poly {poly}')
    else: 
        raise ValueError(f'Specify the date, you passed {date}')
   
   
    r = requests.get(url)
    list_of_dict = json.loads(r.content)
    
    return list_of_dict


In [ ]:
extract_street_level_crimes(lat=51.549740625151905, lng=0.22074769211139364, date='2023-04', crime_type='burglary')

[{'category': 'burglary',
  'location_type': 'Force',
  'location': {'latitude': '51.549919',
   'street': {'id': 1709284, 'name': 'On or near Fernbank Avenue'},
   'longitude': '0.208614'},
  'context': '',
  'outcome_status': {'category': 'Investigation complete; no suspect identified',
   'date': '2023-07'},
  'persistent_id': 'd0b9e31ded76b21f4b5466c871434c61608874f36d35193e5976d32c8a3a5fdb',
  'id': 109591492,
  'location_subtype': '',
  'month': '2023-04'},
 {'category': 'burglary',
  'location_type': 'Force',
  'location': {'latitude': '51.558207',
   'street': {'id': 1710134, 'name': 'On or near Supermarket'},
   'longitude': '0.238649'},
  'context': '',
  'outcome_status': {'category': 'Investigation complete; no suspect identified',
   'date': '2023-07'},
  'persistent_id': 'bd624bb6bd9867e2caa128950267a9f287049a81a0b99d1fde493f69446539a8',
  'id': 109588353,
  'location_subtype': '',
  'month': '2023-04'},
 {'category': 'burglary',
  'location_type': 'Force',
  'location': 

In [2]:
path_to_PAS = '../data/PAS_T%26Cdashboard_to%20Q3%2023-24'

df_PAS_Borough = pd.read_csv(f'{path_to_PAS}_Borough.csv')

df_PAS_Borough = df_PAS_Borough.loc[:, ~df_PAS_Borough.columns.str.contains('^Unnamed')]
df_PAS_Borough = df_PAS_Borough.dropna(axis=1, how='all')

# Preprocess the data for Borough
df_PAS_Borough['Date'] = df_PAS_Borough['Date'].apply(lambda x: x[:7])
df_PAS_Borough['Borough'] = df_PAS_Borough['Borough'].apply(lambda x: 'Westminster' if x == 'City of Westminster' else x)
df_PAS_Borough.loc[df_PAS_Borough['Borough'] == 'Richmond Upon Thames', 'Borough'] = 'Richmond upon Thames'

df_PAS_Borough

,Date,Survey,Borough,Measure,Proportion,MPS
0,2014-12,PAS,Barking and Dagenham,"""Good Job"" local",0.56,0.67
1,2014-12,PAS,Barnet,"""Good Job"" local",0.67,0.67
2,2014-12,PAS,Bexley,"""Good Job"" local",0.72,0.67
3,2014-12,PAS,Brent,"""Good Job"" local",0.66,0.67
4,2014-12,PAS,Bromley,"""Good Job"" local",0.71,0.67
...,...,...,...,...,...,...
9307,2023-12,PAS,Sutton,Understand issues,0.66,0.57
9308,2023-12,PAS,Tower Hamlets,Understand issues,0.55,0.57
9309,2023-12,PAS,Waltham Forest,Understand issues,0.39,0.57
9310,2023-12,PAS,Wandsworth,Understand issues,0.56,0.57


In [7]:
from leafmap import Map
import leafmap as leafmap
from shapely.geometry import Point
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as mcolors


def on_map_click(event, map_instance, df):
    
    if event['type'] == 'click':  # Check for left-click
        lat, lon = event['coordinates']
        print("Latitude:", lat)
        print("Longitude:", lon)

        def in_polygon(polygon):
            point = Point(lon, lat)
            return polygon.contains(point)
        
        # Find the clicked neighborhood
        for index, row in df.iterrows():
            
            if in_polygon(row['geometry']):
                neighborhood_name = row['name']
                coordinates = row['geometry']
                print("Clicked Neighborhood:", neighborhood_name)
                print("Polygon Coordinates:", coordinates)
                break

        

def create_map():
    # Create a map centered around London
    m = Map(center=(51.5074, -0.1278), zoom=10)

    # Load the geojson file for the boroughs
    neighbourhoods = gpd.read_file('../data/neighbourhoods_boundary.geojson')

    # Load the data for the specific date and measure
    filtered_data = df_PAS_Borough[(df_PAS_Borough['Date'].str.contains(str('2023-12'))) & (df_PAS_Borough['Measure'] == 'Understand issues')]

    # Merge the filtered data with the boroughs data
    merged_data = gpd.GeoDataFrame(neighbourhoods.merge(filtered_data, left_on='borough', right_on='Borough'))

    def proportional_color(feature):
        # Get the value indicating proportion from the DataFrame
        proportion_value = feature['properties']['Proportion']

        # Define a colormap
        cmap = LinearSegmentedColormap.from_list('proportional_color', ['red', 'yellow', 'green'])
        
        # Map the proportion value to a color
        color = cmap(proportion_value)

        # Convert the color to HTML color code
        html_color = mcolors.rgb2hex(color)
        
        return {
            'color': 'black',
            'fillColor': html_color,
        }

    # Add a GeoJson layer to the map
    geojson_layer = leafmap.geojson_layer(
        in_geojson=merged_data._to_geo(), 
        style={'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
        hover_style={'fillColor': 'red'},
        style_callback=proportional_color,
    )
    
    m.add_layer(geojson_layer)
    
    # Add a click event handler to the map
    def click_handler(event=None, **kwargs):
        clicked = on_map_click(kwargs, m, merged_data)
        if clicked is not None:
            print("Clicked points:", clicked)

    m.on_interaction(click_handler)

    # Display the map
    return m

create_map()

DeltaGenerator()

In [ ]:
def extract_street_level_outcomes(location_id: int = None, lat: float = None, lng: float = None, date: str = None, poly: list = None):
    """
    Params:
    lat - Latitude of the requested crime area
    lng - Longitude of the requested crime area
    date - Optional. "YYYY-MM" Limit results to a specific month. 
        The latest month will be shown by default
    poly - The lat/lng pairs which define the boundary of the custom area. 
        The poly parameter is formatted in lat/lng pairs, separated by 
        colons: [lat],[lng]:[lat],[lng]:[lat],[lng]
    location_id - Crimes and outcomes are mapped to specific locations on the map. 
        Valid IDs are returned by other methods which return location information.
    """

    # Get current year
    current_year = datetime.now().year

    # Get current month
    current_month = datetime.now().month

    base_url = 'https://data.police.uk/api/outcomes-at-location?'

    if date == None:
        if lat and lng != None:
            url = base_url + f"date={current_year}-{current_month}&lat={lat}&lng={lng}"
        elif poly != None:
            url = base_url + f"date={current_year}-{current_month}&poly={poly[0][0]}:{poly[0][1]},{poly[1][0]}:{poly[1][1]},{poly[2][0]},{poly[2][1]}"
        elif location_id != None:
            url = base_url + f"date={current_year}-{current_month}&location_id={location_id}"
        else:
            raise ValueError(f'Specify the lat/lang or poly or location_id that you passed or make sure they are correct: location_id-{location_id}, lat/lang-{lat, lng}, poly={poly}')
    
    elif date != None:
        if lat and lng != None:
            url = base_url + f"date={date}&lat={lat}&lng={lng}"
        elif poly != None:
            url = base_url + f"date={date}&poly={poly[0][0]}:{poly[0][1]},{poly[1][0]}:{poly[1][1]},{poly[2][0]}:{poly[2][1]}"
        elif location_id != None:
            url = base_url + f"date={date}&location_id={location_id}"
        else:
            raise ValueError(f'Specify the lat/lang or poly or location_id that you passed or make sure they are correct: location_id-{location_id}, lat/lang-{lat, lng}, poly={poly}')
        
    else: 
        raise ValueError(f'Specify the date, you passed {date}')
   
   
    r = requests.get(url)
    list_of_dict = json.loads(r.content)
    
    return list_of_dict


In [ ]:
extract_street_level_outcomes(location_id=1709052, date='2023-04')

[{'category': {'code': 'no-further-action',
   'name': 'Investigation complete; no suspect identified'},
  'date': '2023-04',
  'person_id': None,
  'crime': {'category': 'violent-crime',
   'location_type': 'Force',
   'location': {'latitude': '51.548660',
    'street': {'id': 1709052, 'name': ' '},
    'longitude': '0.199232'},
   'context': '',
   'persistent_id': '99153a619aa37e592722b1d58bcc742ae08042b18bb944b731b0ad595bc19ec9',
   'id': 103333792,
   'location_subtype': 'SUPERMARKET CHAINS',
   'month': '2022-07'}},
 {'category': {'code': 'no-further-action',
   'name': 'Investigation complete; no suspect identified'},
  'date': '2023-04',
  'person_id': None,
  'crime': {'category': 'violent-crime',
   'location_type': 'Force',
   'location': {'latitude': '51.548660',
    'street': {'id': 1709052, 'name': ' '},
    'longitude': '0.199232'},
   'context': '',
   'persistent_id': '99153a619aa37e592722b1d58bcc742ae08042b18bb944b731b0ad595bc19ec9',
   'id': 103333792,
   'location_s

In [ ]:
def extract_crimes_at_location(location_id: int = None, lat: float = None, lng: float = None, date: str = None):
    """
    Params:
    lat - Latitude of the requested crime area
    lng - Longitude of the requested crime area
    date - Optional. "YYYY-MM" Limit results to a specific month. 
        The latest month will be shown by default
    location_id - Crimes and outcomes are mapped to specific locations on the map. 
        Valid IDs are returned by other methods which return location information.
    """

    # Get current year
    current_year = datetime.now().year

    # Get current month
    current_month = datetime.now().month

    base_url = 'https://data.police.uk/api/crimes-at-location?'

    if date == None:
        if lat and lng != None:
            url = base_url + f"date={current_year}-{current_month}&lat={lat}&lng={lng}"
        elif location_id != None:
            url = base_url + f"date={current_year}-{current_month}&location_id={location_id}"
        else:
            raise ValueError(f'Specify the lat/lang or location_id that you passed or make sure they are correct: location_id-{location_id}, lat/lang-{lat, lng}')
    
    elif date != None:
        if lat and lng != None:
            url = base_url + f"date={date}&lat={lat}&lng={lng}"
        elif location_id != None:
            url = base_url + f"date={date}&location_id={location_id}"
        else:
            raise ValueError(f'Specify the lat/lang or location_id that you passed or make sure they are correct: location_id-{location_id}, lat/lang-{lat, lng}')
        
    else: 
        raise ValueError(f'Specify the date, you passed {date}')
   
   
    r = requests.get(url)
    list_of_dict = json.loads(r.content)
    
    return list_of_dict


In [ ]:
extract_crimes_at_location(location_id=1709052, date='2023-04')

[{'category': 'anti-social-behaviour',
  'location_type': 'Force',
  'location': {'latitude': '51.548660',
   'street': {'id': 1709052, 'name': 'On or near '},
   'longitude': '0.199232'},
  'context': '',
  'outcome_status': None,
  'persistent_id': '',
  'id': 109492434,
  'location_subtype': '',
  'month': '2023-04'},
 {'category': 'burglary',
  'location_type': 'Force',
  'location': {'latitude': '51.548660',
   'street': {'id': 1709052, 'name': 'On or near Supermarket'},
   'longitude': '0.199232'},
  'context': '',
  'outcome_status': {'category': 'Investigation complete; no suspect identified',
   'date': '2023-07'},
  'persistent_id': '1c0edf9429e38153e8e89c88cc339da171e14520787ffef294e8bb90e035559e',
  'id': 109588351,
  'location_subtype': '',
  'month': '2023-04'},
 {'category': 'public-order',
  'location_type': 'Force',
  'location': {'latitude': '51.548660',
   'street': {'id': 1709052, 'name': 'On or near '},
   'longitude': '0.199232'},
  'context': '',
  'outcome_statu

In [ ]:
def extract_crimes_no_location(category: str = 'all-crime', force: str = 'leicestershire', date: str = None):
    """
    Params:
    category - The type of crime. Options are: None, anti-social-behaviour, burglary
    date - Optional. "YYYY-MM" Limit results to a specific month. 
        The latest month will be shown by default
    location_id - Crimes and outcomes are mapped to specific locations on the map. 
        Valid IDs are returned by other methods which return location information.
    """

    # Get current year
    current_year = datetime.now().year

    # Get current month
    current_month = datetime.now().month

    base_url = 'https://data.police.uk/api/crimes-no-location?'

    if date == None:
        if (category and force) != None:
            url = base_url + f"category={category}&force={force}&date={current_year}-{current_month}"
        else:
            raise ValueError(f'Specify the category and force that you passed or make sure they are correct: category-{category}, force-{force}')
    
    elif date != None:
        if (category and force) != None:
            url = base_url + f"category={category}&force={force}&date={date}"
        else:
            raise ValueError(f'Specify the category and force that you passed or make sure they are correct: category-{category}, force-{force}')
    
    else: 
        raise ValueError(f'Specify the date, you passed: date={date}')
   
   
    r = requests.get(url)
    list_of_dict = json.loads(r.content)
    
    return list_of_dict


In [ ]:
extract_crimes_no_location(category='all-crime', date='2024-01', force='leicestershire')

[{'category': 'bicycle-theft',
  'location_type': None,
  'location': None,
  'context': '',
  'outcome_status': {'category': 'Investigation complete; no suspect identified',
   'date': '2024-02'},
  'persistent_id': '3cce1d68d447e230f4d341552260b75a444355d2c2dc89052363a235b9b43902',
  'id': 116203234,
  'location_subtype': '',
  'month': '2024-01'},
 {'category': 'bicycle-theft',
  'location_type': None,
  'location': None,
  'context': '',
  'outcome_status': {'category': 'Investigation complete; no suspect identified',
   'date': '2024-02'},
  'persistent_id': '2dfbe17ccd04260ea94db37ae5d759078cd728430c668c84b5b8b1a08ec253ca',
  'id': 116205668,
  'location_subtype': '',
  'month': '2024-01'},
 {'category': 'burglary',
  'location_type': None,
  'location': None,
  'context': '',
  'outcome_status': {'category': 'Under investigation', 'date': '2024-01'},
  'persistent_id': '1435974b6758d2d3be4da594a84913877173485d6a39c70a25841f1e1ce28459',
  'id': 116206380,
  'location_subtype': ''

In [ ]:
def extract_crimes_categories_at_date(date: str = None):
    """
    Params:
    date - Optional. "YYYY-MM" Limit results to a specific month. 
        The latest month will be shown by default
    """

    # Get current year
    current_year = datetime.now().year

    # Get current month
    current_month = datetime.now().month

    base_url = 'https://data.police.uk/api/crime-categories?'

    if date == None:
        
        url = base_url + f"date={current_year}-{current_month}"
        
    elif date != None:
        
        url = base_url + f"date={date}"
       
    else: 
        raise ValueError(f'Specify the date, you passed {date}')
   
   
    r = requests.get(url)
    list_of_dict = json.loads(r.content)
    
    return list_of_dict


In [ ]:
extract_crimes_categories_at_date(date='2023-04')

[{'url': 'all-crime', 'name': 'All crime'},
 {'url': 'anti-social-behaviour', 'name': 'Anti-social behaviour'},
 {'url': 'bicycle-theft', 'name': 'Bicycle theft'},
 {'url': 'burglary', 'name': 'Burglary'},
 {'url': 'criminal-damage-arson', 'name': 'Criminal damage and arson'},
 {'url': 'drugs', 'name': 'Drugs'},
 {'url': 'other-theft', 'name': 'Other theft'},
 {'url': 'possession-of-weapons', 'name': 'Possession of weapons'},
 {'url': 'public-order', 'name': 'Public order'},
 {'url': 'robbery', 'name': 'Robbery'},
 {'url': 'shoplifting', 'name': 'Shoplifting'},
 {'url': 'theft-from-the-person', 'name': 'Theft from the person'},
 {'url': 'vehicle-crime', 'name': 'Vehicle crime'},
 {'url': 'violent-crime', 'name': 'Violence and sexual offences'},
 {'url': 'other-crime', 'name': 'Other crime'}]

In [ ]:
def extract_last_updated():
    
    url = 'https://data.police.uk/api/crime-last-updated'

    r = requests.get(url)
    list_of_dict = json.loads(r.content)
    
    return list_of_dict

In [ ]:
extract_last_updated()

{'date': '2024-03-01'}

In [ ]:
def extract_outcome_for_specific_crime(crime_id: str = None):
    """
    Returns the outcomes (case history) for the specified crime.
    Note: Outcomes are not available for the Police Service of Northern Ireland.

    Parameters:
    crime_id is 64-character identifier
    """

    base_url = 'https://data.police.uk/api/outcomes-for-crime/'

    if crime_id != None:
        url = base_url + f"{crime_id}"
    elif crime_id == None:
        raise ValueError(f"Specify the crime_id that you passed and make sure it's correct: crime_id={crime_id}")
    
    r = requests.get(url)
    list_of_dict = json.loads(r.content)

    return list_of_dict


In [ ]:
extract_outcome_for_specific_crime(crime_id='1c0edf9429e38153e8e89c88cc339da171e14520787ffef294e8bb90e035559e')

{'outcomes': [{'category': {'code': 'under-investigation',
    'name': 'Under investigation'},
   'date': '2023-04',
   'person_id': None},
  {'category': {'code': 'no-further-action',
    'name': 'Investigation complete; no suspect identified'},
   'date': '2023-07',
   'person_id': None}],
 'crime': {'category': 'burglary',
  'location_type': 'Force',
  'location': {'latitude': '51.548660',
   'street': {'id': 1709052, 'name': 'On or near Supermarket'},
   'longitude': '0.199232'},
  'context': '',
  'persistent_id': '1c0edf9429e38153e8e89c88cc339da171e14520787ffef294e8bb90e035559e',
  'id': 109588351,
  'location_subtype': '',
  'month': '2023-04'}}

# Neighbourhood related

In [ ]:
def extract_neighbourhoods_for_a_force(force: str = 'leicestershire'):
    """
    Returns the neighbourhoods for the specified force.
    Parameters:
    force - string that represents the name of force (default: 'leicestershire')
    """

    if force != None:
        url = f'https://data.police.uk/api/{force}/neighbourhoods'

    elif force == None:
        raise ValueError(f"Specify the force that you passed and make sure it's correct: force={force}")
    
    r = requests.get(url)
    list_of_dict = json.loads(r.content)

    return list_of_dict

In [ ]:
extract_neighbourhoods_for_a_force(force='leicestershire')

[{'id': 'NC04', 'name': 'City Centre'},
 {'id': 'NC66', 'name': 'Cultural Quarter'},
 {'id': 'NC67', 'name': 'Riverside'},
 {'id': 'NC68', 'name': 'Clarendon Park'},
 {'id': 'NE09', 'name': 'Latimer'},
 {'id': 'NE10', 'name': 'Belgrave'},
 {'id': 'NE11', 'name': 'Rushey Mead'},
 {'id': 'NE12', 'name': 'Humberstone and Hamilton'},
 {'id': 'NE13', 'name': 'Northfields, Tailby and Morton'},
 {'id': 'NE14', 'name': 'Thurncourt'},
 {'id': 'NE15', 'name': 'Stoneygate'},
 {'id': 'NE16', 'name': 'Spinney Hills'},
 {'id': 'NE17', 'name': 'Evington'},
 {'id': 'NE18', 'name': 'Coleman'},
 {'id': 'NH19', 'name': 'Thorpe Astley and Braunstone Town'},
 {'id': 'NH20', 'name': 'Enderby, Narborough, Littlethorpe and Fosse Park'},
 {'id': 'NH21', 'name': 'Blaby, Whetstone, Glen Parva and Cosby'},
 {'id': 'NH22', 'name': 'Leicester Forest East, Kirby Muxloe and Glenfield'},
 {'id': 'NH23', 'name': 'Countesthorpe, Foston and Kilby'},
 {'id': 'NH24', 'name': 'Fosse Villages'},
 {'id': 'NH25', 'name': 'Hinc

In [ ]:
def extract_specific_neighbourhoods_for_a_force(force: str = 'leicestershire', neighbourhood_id: str = 'NC04'):
    base_url = 'https://data.police.uk/api/'

    if (force != None) and (neighbourhood_id != None):
        url = base_url + f"{force}/{neighbourhood_id}"
    elif (force == None) and (neighbourhood_id == None):
        return f'specify the neighbourhood_id and force'
    else:
        raise ValueError(f"Specify the force and neighbourhood that you passed and make sure it's correct: force={force}, neighbourhood_id={neighbourhood_id}")
    
    r = requests.get(url)
    list_of_dict = json.loads(r.content)

    return list_of_dict

In [ ]:
extract_specific_neighbourhoods_for_a_force(force='leicestershire', neighbourhood_id='NC04')

{'url_force': 'http://www.leics.police.uk/local-policing/city-centre',
 'contact_details': {},
 'name': 'City Centre',
 'links': [{'url': 'http://www.leicester.gov.uk/',
   'description': None,
   'title': 'Leicester City Council'}],
 'centre': {'latitude': '52.6389', 'longitude': '-1.13619'},
 'locations': [],
 'description': "<p>The Castle neighbourhood\xa0is\xa0a diverse\xa0covering all of the City Centre. In addition it covers De Montfort University, the University of Leicester,\xa0Leicester Royal Infirmary, the Leicester Tigers rugby ground and\xa0the Clarendon Park and Riverside communities.</p>\n<p>The Highcross and Haymarket shopping centres\xa0and Leicester's famous Market are all covered by\xa0this neighbourhood.</p>",
 'id': 'NC04',
 'population': '0'}

In [ ]:
def extract_neighbourhood_boundary_for_a_force(force: str = 'leicestershire', neighbourhood_id: str = 'NC04'):
    base_url = 'https://data.police.uk/api/'

    if (force != None) and (neighbourhood_id != None):
        url = base_url + f"{force}/{neighbourhood_id}/boundary"
    elif (force == None) and (neighbourhood_id == None):
        return f'specify the neighbourhood_id and force'
    else:
        raise ValueError(f"Specify the force and neighbourhood that you passed and make sure it's correct: force={force}, neighbourhood_id={neighbourhood_id}")
    
    r = requests.get(url)
    list_of_dict = json.loads(r.content)

    return list_of_dict

In [ ]:
extract_neighbourhood_boundary_for_a_force(force='leicestershire', neighbourhood_id = 'NC04')

[{'latitude': '52.6394052587', 'longitude': '-1.1458618876'},
 {'latitude': '52.6389452755', 'longitude': '-1.1457057759'},
 {'latitude': '52.6383706746', 'longitude': '-1.1455755443'},
 {'latitude': '52.637738214', 'longitude': '-1.1453756586'},
 {'latitude': '52.6372925229', 'longitude': '-1.1452192785'},
 {'latitude': '52.6369615308', 'longitude': '-1.1450608136'},
 {'latitude': '52.6365882521', 'longitude': '-1.1449974537'},
 {'latitude': '52.6361567723', 'longitude': '-1.1448171605'},
 {'latitude': '52.6354495167', 'longitude': '-1.1441945098'},
 {'latitude': '52.6347986756', 'longitude': '-1.1434293643'},
 {'latitude': '52.6341926728', 'longitude': '-1.14292269'},
 {'latitude': '52.6338167018', 'longitude': '-1.1424375394'},
 {'latitude': '52.6338780456', 'longitude': '-1.1418829669'},
 {'latitude': '52.6338773726', 'longitude': '-1.141790037'},
 {'latitude': '52.6338766792', 'longitude': '-1.1416942999'},
 {'latitude': '52.6340579308', 'longitude': '-1.1415716571'},
 {'latitude'

In [ ]:
def extract_neighbourhood_team_for_a_force(force: str = 'leicestershire', neighbourhood_id: str = 'NC04'):
    base_url = 'https://data.police.uk/api/'

    if (force != None) and (neighbourhood_id != None):
        url = base_url + f"{force}/{neighbourhood_id}/people"
    elif (force == None) and (neighbourhood_id == None):
        return f'specify the neighbourhood_id and force'
    else:
        raise ValueError(f"Specify the force and neighbourhood that you passed and make sure it's correct: force={force}, neighbourhood_id={neighbourhood_id}")
    
    r = requests.get(url)
    list_of_dict = json.loads(r.content)

    return list_of_dict

In [ ]:
extract_neighbourhood_team_for_a_force(force = 'leicestershire', neighbourhood_id = 'NC04')

[{'bio': None,
  'contact_details': {},
  'name': 'Zane Tompkins',
  'rank': 'Sergeant 4081'},
 {'bio': None,
  'contact_details': {},
  'name': 'Bec Sanders',
  'rank': 'PCSO 6628'},
 {'bio': None,
  'contact_details': {},
  'name': 'Donna Green',
  'rank': 'PCSO 6669'},
 {'bio': None,
  'contact_details': {},
  'name': 'Jane Elton',
  'rank': 'PCSO 6040'},
 {'bio': None,
  'contact_details': {},
  'name': 'Tim Jones',
  'rank': 'PCSO 6036'},
 {'bio': None,
  'contact_details': {},
  'name': 'Karl Turner',
  'rank': 'Police Constable 1467'},
 {'bio': None,
  'contact_details': {},
  'name': 'Tom Page-Brown',
  'rank': 'Police Constable 4486'},
 {'bio': None,
  'contact_details': {},
  'name': 'Jordan Spencer',
  'rank': 'Police Constable 4611'},
 {'bio': None,
  'contact_details': {},
  'name': 'Katie Spencer',
  'rank': 'Police Constable 4416'},
 {'bio': None,
  'contact_details': {},
  'name': 'Elliot Godden',
  'rank': 'Police Constable 4323'},
 {'bio': None,
  'contact_details': {

In [ ]:
def extract_neighbourhood_events_for_a_force(force: str = 'leicestershire', neighbourhood_id: str = 'NC04'):
    base_url = 'https://data.police.uk/api/'

    if (force != None) and (neighbourhood_id != None):
        url = base_url + f"{force}/{neighbourhood_id}/events"
    elif (force == None) and (neighbourhood_id == None):
        return f'specify the neighbourhood_id and force'
    else:
        raise ValueError(f"Specify the force and neighbourhood that you passed and make sure it's correct: force={force}, neighbourhood_id={neighbourhood_id}")
    
    r = requests.get(url)
    list_of_dict = json.loads(r.content)

    return list_of_dict

In [ ]:
extract_neighbourhood_events_for_a_force(force = 'leicestershire', neighbourhood_id = 'NC04')

[{'contact_details': {},
  'description': '<p>PCSO Jane Elton and PC Tom Page-Brown, will be at the Abbey park Campus, registering students and staffs bikes to the bike register. This is a completely free service, Dr Bike will also be there to fix any minor issues with your bike.</p>',
  'end_date': '2024-05-14T14:00:00',
  'title': 'Dr Bike, Bike Register Event, Abbey Park Campus',
  'address': 'Abbey Park Campus, Leicester ',
  'type': 'other',
  'start_date': '2024-05-14T10:00:00'},
 {'contact_details': {},
  'description': '<p>PCSO Tim Jones will be conducting a series of visits to stores ands chats with visitors about various crime reduction themes including thefts and burglaries</p>',
  'end_date': '2024-05-16T18:00:00',
  'title': 'Crime reduction drop-in, Haymarket Shopping Centre',
  'address': 'Haymarket Shopping Centre, Leicester',
  'type': 'other',
  'start_date': '2024-05-16T16:30:00'},
 {'contact_details': {},
  'description': '<p>PCSO Donna Green, will be at the Blood D

In [ ]:
def extract_neighbourhood_priorities_for_a_force(force: str = 'leicestershire', neighbourhood_id: str = 'NC04'):
    base_url = 'https://data.police.uk/api/'

    if (force != None) and (neighbourhood_id != None):
        url = base_url + f"{force}/{neighbourhood_id}/priorities"
    elif (force == None) and (neighbourhood_id == None):
        return f'specify the neighbourhood_id and force'
    else:
        raise ValueError(f"Specify the force and neighbourhood that you passed and make sure it's correct: force={force}, neighbourhood_id={neighbourhood_id}")
    
    r = requests.get(url)
    list_of_dict = json.loads(r.content)

    return list_of_dict

In [ ]:
extract_neighbourhood_priorities_for_a_force(force = 'leicestershire', neighbourhood_id = 'NC04')

[{'action': '<p>*APRIL UPDATE*<br />WE DID:<br />Joint patrols conducted with Highcross security focusing on shop theft and anti-social behaviour.<br />Visits to Vision Express and Cotswold Outdoor to offer tailored advice to management on the topic of shop theft.<br />A male suspect identified for burglary at the Highcross has been arrested, charged and remanded.<br />A male suspect trying to sell stolen items in the City Centre was caught in the act by the beat team. The items have been returned to the owner and the male has been processed for the offence via an out of court disposal.<br />A prolific shop thief has been issued with a CBO banning him from various locations in the city centre after being charged with multiple theft offences.<br />Our PCSO’s have held various engagements across the city centre focusing on the subject of crime prevention, including theft. Please check our website for our upcoming events.</p>',
  'issue-date': '2024-04-01T00:00:00',
  'action-date': '2024

In [ ]:
def extract_neighbourhood_at_location(lat: float, lng: float):
    base_url = 'https://data.police.uk/api/locate-neighbourhood?'

    if (lat != None) and (lng != None):
        url = base_url + f"q={lat},{lng}"
    else:
        return f'specify the lat and lng'
   
    r = requests.get(url)
    list_of_dict = json.loads(r.content)

    return list_of_dict

In [ ]:
extract_neighbourhood_at_location(lat=51.500617, lng=-0.124629)

{'force': 'metropolitan', 'neighbourhood': 'E05013806N'}

# Stop and Search

In [ ]:
def extract_stop_search(date: str = None, lng: float = None, lat: float = None, poly: list = None, location_id: str = None):
    """
    Params:
    lat - Latitude of the requested crime area
    lng - Longitude of the requested crime area
    date - Optional. "YYYY-MM" Limit results to a specific month. 
        The latest month will be shown by default
    poly - The lat/lng pairs which define the boundary of the custom area. 
        The poly parameter is formatted in lat/lng pairs, separated by 
        colons: [lat],[lng]:[lat],[lng]:[lat],[lng]
    location_id - The ID of the location to get stop and searches for
    """

    # Get current year
    current_year = datetime.now().year

    # Get current month
    current_month = datetime.now().month
    if location_id != None:
        base_url = "http://data.police.uk/api/stops-at-location?"
    else: 
        base_url = 'https://data.police.uk/api/stops-street?'

    if date == None:
        if lat and lng != None:
            url = base_url + f"lat={lat}&lng={lng}&date={current_year}-{current_month}"
        elif poly != None:
            url = base_url + f"poly={poly[0][0]}:{poly[0][1]},{poly[1][0]}:{poly[1][1]},{poly[2][0]},{poly[2][1]}&date={current_year}-{current_month}"
        elif location_id!= None:
            url = base_url + f"location_id={location_id}&date={current_year}-{current_month}"
        else:
            raise ValueError(f'Specify the lat/lang or poly that you passed or make sure they are correct: lat/lang-{lat, lng}, poly={poly}')
    
    elif date != None:
        if lat and lng != None:
            url = base_url + f"lat={lat}&lng={lng}&date={date}"
        elif poly != None:
            url = base_url + f"poly={poly[0][0]}:{poly[0][1]},{poly[1][0]}:{poly[1][1]},{poly[2][0]}:{poly[2][1]}&date={date}"
        elif location_id!= None:
            url = base_url + f"location_id={location_id}&date={date}"
        else:
            raise ValueError(f'Specify the lat/lang or poly that you passed or make sure they are correct: lat/lang-{lat, lng}, poly={poly}')
        
    else: 
        raise ValueError(f'Specify the date, you passed {date}')
   
   
    r = requests.get(url)
    list_of_dict = json.loads(r.content)
    
    return list_of_dict

In [ ]:
extract_stop_search(date = '2023-04', lat = 51.56768064709022, lng = -0.19088745117187503)

[{'age_range': '10-17',
  'outcome': 'Arrest',
  'involved_person': True,
  'self_defined_ethnicity': 'Black/African/Caribbean/Black British - Caribbean',
  'gender': 'Male',
  'legislation': 'Firearms Act 1968 (section 47)',
  'outcome_linked_to_object_of_search': None,
  'datetime': '2023-04-09T23:57:00+00:00',
  'removal_of_more_than_outer_clothing': None,
  'outcome_object': {'id': 'bu-arrest', 'name': 'Arrest'},
  'location': {'latitude': '51.559283',
   'street': {'id': 1671125, 'name': 'On or near Windmill Hill'},
   'longitude': '-0.180776'},
  'operation': False,
  'officer_defined_ethnicity': 'Black',
  'type': 'Person search',
  'operation_name': None,
  'object_of_search': 'Firearms'},
 {'age_range': 'over 34',
  'outcome': 'A no further action disposal',
  'involved_person': True,
  'self_defined_ethnicity': 'White - Any other White background',
  'gender': 'Male',
  'legislation': 'Police and Criminal Evidence Act 1984 (section 1)',
  'outcome_linked_to_object_of_search':

In [ ]:
def extract_stop_search_noloc(date: str, force: str = None):
    """
    Parameters:
    date - Optional. "YYYY-MM" Limit results to a specific month.
    The latest month will be shown by default
    force - the name of the forces
    """

    base_url = 'https://data.police.uk/api/stops-no-location?'

    # Get current year
    current_year = datetime.now().year

    # Get current month
    current_month = datetime.now().month

    if date == None:
        if force != None:
            url = base_url + f"force={force}&date={current_year}-{current_month}"
        else:
            raise ValueError(f'Specify the name of the force parameter')
    
    elif date != None:
        if force != None:
            url = base_url + f"force={force}&date={date}"
        else:
            raise ValueError(f'Specify the name of the force parameter')
        
    else: 
        raise ValueError(f'Specify the date, you passed {date}')
   
   
    r = requests.get(url)
    list_of_dict = json.loads(r.content)
    
    return list_of_dict


In [ ]:
extract_stop_search_noloc(date='2023-02', force='cleveland')

[{'age_range': 'over 34',
  'outcome': 'A no further action disposal',
  'involved_person': True,
  'self_defined_ethnicity': 'White - English/Welsh/Scottish/Northern Irish/British',
  'gender': 'Male',
  'legislation': 'Misuse of Drugs Act 1971 (section 23)',
  'outcome_linked_to_object_of_search': True,
  'datetime': '2023-02-04T00:00:00+00:00',
  'removal_of_more_than_outer_clothing': None,
  'outcome_object': {'id': 'bu-no-further-action',
   'name': 'A no further action disposal'},
  'location': None,
  'operation': None,
  'officer_defined_ethnicity': 'White',
  'type': 'Person search',
  'operation_name': None,
  'object_of_search': 'Controlled drugs'},
 {'age_range': '25-34',
  'outcome': 'A no further action disposal',
  'involved_person': True,
  'self_defined_ethnicity': None,
  'gender': 'Female',
  'legislation': 'Misuse of Drugs Act 1971 (section 23)',
  'outcome_linked_to_object_of_search': None,
  'datetime': '2023-02-04T00:00:00+00:00',
  'removal_of_more_than_outer_cl

In [ ]:
def extract_stop_search_force(date: str = None, force: str = 'avon-and-somerset'):
    """
    Parameters:
    date - Optional. "YYYY-MM" Limit results to a specific month.
    The latest month will be shown by default
    force - the name of the forces
    """

    base_url = 'https://data.police.uk/api/stops-force?'

    # Get current year
    current_year = datetime.now().year

    # Get current month
    current_month = datetime.now().month

    if date == None:
        if force != None:
            url = base_url + f"force={force}&date={current_year}-{current_month}"
        else:
            raise ValueError(f'Specify the name of the force parameter')
    
    elif date != None:
        if force != None:
            url = base_url + f"force={force}&date={date}"
        else:
            raise ValueError(f'Specify the name of the force parameter')
        
    else: 
        raise ValueError(f'Specify the date, you passed {date}')
   
   
    r = requests.get(url)
    list_of_dict = json.loads(r.content)
    
    return list_of_dict

In [ ]:
extract_stop_search_force(date='2023-04', force='avon-and-somerset')

[{'age_range': '25-34',
  'outcome': '',
  'involved_person': True,
  'self_defined_ethnicity': 'White - English/Welsh/Scottish/Northern Irish/British',
  'gender': 'Male',
  'legislation': 'Misuse of Drugs Act 1971 (section 23)',
  'outcome_linked_to_object_of_search': None,
  'datetime': '2023-04-04T00:00:00+00:00',
  'removal_of_more_than_outer_clothing': False,
  'outcome_object': {'id': '', 'name': ''},
  'location': {'latitude': '51.392086',
   'street': {'id': 2288310, 'name': 'On or near Cavendish Crescent'},
   'longitude': '-2.369735'},
  'operation': None,
  'officer_defined_ethnicity': 'White',
  'type': 'Person search',
  'operation_name': None,
  'object_of_search': 'Controlled drugs'},
 {'age_range': 'over 34',
  'outcome': 'Community resolution',
  'involved_person': True,
  'self_defined_ethnicity': 'White - English/Welsh/Scottish/Northern Irish/British',
  'gender': 'Male',
  'legislation': 'Police and Criminal Evidence Act 1984 (section 1)',
  'outcome_linked_to_obje